# Environment
- GPU : NVIDIA GeForce RTX 3070
- CPU : 12th Gen Intel(R) Core(TM) i5-12600K

## Library version check

In [2]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
from pandas import Timestamp
from datetime import datetime
from datetime import timedelta
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
pandas version: 2.1.2
numpy version: 1.23.5
matplotlib version: 3.5.2
tqdm version: 4.65.0
sktime version: 0.20.1
xgboost version: 1.7.6
seaborn version: 0.11.2
scikit-learn version: 1.0.2
------------------------------------------------------------------------------


In [3]:
import matplotlib.pyplot as plt
#시각화 설정
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 15
# 사용자 운영체제 확인
import os
os.name

# 운영체제별 한글 폰트 설정
if os.name == 'posix': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

## 0. load the libararies

In [4]:
import pickle

In [5]:
import bisect
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold , KFold
import matplotlib.pyplot as plt
from tqdm import tqdm
from xgboost import XGBRegressor
import warnings

pd.set_option('display.max_columns', 200)

# 후처리 방향

train data의 날씨 컬럼이 도착기준 예보날짜이기 때문에 이를 통해 어느정도 날씨의 정보를 이용해 도착 예상 날짜를 예측할 수 있다.  
Dacon 측에서 train에 해당하는 데이터 모두 추론시점에서 이미 다 알고있다는 가정하에 진행하였으므로 test가 어떤 것이 들어오는 모든 data를 사용가능하다.  

train data에서 ATA에 target값인 CI_HOUR 를 더해 bert_date 즉 도착했을 때의 시점을 만들고 나라, 항구, 날씨정보들을 통해 도착예상 날짜들을  train 기반으로 생성하고 이를 test에 merge한 후 해당 test의 ata에 따라 train에서 얻은 정보 중에 사용할 train데이터를 선정해 활용하는 방식.

train data를 통해 얻은 정보를 test에 merge하는 과정 = data_leakage 가 아니다. test시점은 train이 모두 일어난 후로 본 대회에서 정의.  
test에서 test 정보에 따라 위의 정보를 선정하는 과정 = data_leakage 가 아니다. test가 어떤 방식으로 들어오든 처리가 가능하고, test 내에서의 열연산 이기 때문에 가능하다.

train data에서 bert_date를 만들때 최대 값과 최소 값을 비교하여 이가 240 즉 10일 이상 차이날 경우 평균을 사용하는 데에 위험부담이 있기 때문에 min값만을 사용하고 diff가 240보다 작다면 mean값을 사용한다.

## 1. Load Data

In [6]:
train = pd.read_csv('../data/train.csv')#.drop(columns=['SAMPLE_ID'])
test = pd.read_csv('../data/test.csv')#.drop(columns=['SAMPLE_ID'])

In [7]:
submission = pd.read_csv('../submission/sample_submission.csv')#.drop(columns=['SAMPLE_ID'])

null값이 없는 데이터만 선택. air_temp 가 nan인 값도 의미가 있다고 판단해 처리하지 않음.

In [8]:
train_weather = train[train['U_WIND'].notnull()].copy()
test_weather = test[test['U_WIND'].notnull()].copy()

In [9]:
train_weather['ATA'] = pd.to_datetime(train_weather['ATA'])  # 'ATA' 열을 날짜 및 시간 형식으로 변환
train_weather['bert_date'] = train_weather['ATA'] + pd.to_timedelta(train_weather['CI_HOUR'], unit="hours")
train_weather['bert_date_date'] = train_weather['bert_date'].dt.floor('D').astype('str')
train_weather['bert_year'] = train_weather['bert_date'].dt.year
train_weather['bert_month'] = train_weather['bert_date'].dt.month
train_weather['bert_day'] = train_weather['bert_date'].dt.day
train_weather['bert_hour'] = train_weather['bert_date'].dt.hour
train_weather['bert_minute'] = train_weather['bert_date'].dt.minute

In [10]:
train_weather['check_bert'] = train_weather['ARI_CO'].astype(str) + '_' + train_weather['ARI_PO'].astype(str) + '_' + train_weather['U_WIND'].astype(str) + '_' + train_weather['V_WIND'].astype(str) + '_' + train_weather['AIR_TEMPERATURE'].astype(str) + '_' + train_weather['BN'].astype(str)
test_weather['check_bert'] = test_weather['ARI_CO'].astype(str) + '_' + test_weather['ARI_PO'].astype(str) + '_' + test_weather['U_WIND'].astype(str) + '_' + test_weather['V_WIND'].astype(str) + '_' + test_weather['AIR_TEMPERATURE'].astype(str) + '_' + test_weather['BN'].astype(str)

In [11]:
Check_Bert = pd.DataFrame(train_weather.groupby('check_bert')['bert_date'].max()).reset_index()

In [12]:
Check_Bert.columns = ['check_bert', 'max_ata']

In [13]:
tmp = pd.DataFrame(train_weather.groupby('check_bert')['bert_date'].min()).reset_index()
tmp.columns = ['check_bert', 'min_ata']
Check_Bert = Check_Bert.merge(tmp, on='check_bert')

In [14]:
tmp = pd.DataFrame(train_weather.groupby('check_bert')['bert_date'].count()).reset_index()
tmp.columns = ['check_bert', 'count_ata']
Check_Bert = Check_Bert.merge(tmp, on='check_bert')

In [15]:
Check_Bert['diff_ata'] = (Check_Bert['max_ata'] - Check_Bert['min_ata']).dt.total_seconds()/ 3600

***

위에서 정의했던 240 즉 10일을 기준으로 데이터를 분리 및 따로 처리

In [16]:
Check_Bert_under10 = Check_Bert[Check_Bert['diff_ata'] < 240].copy()

In [17]:
Check_Bert_over10 = Check_Bert[Check_Bert['diff_ata'] >= 240].copy()

***

# 🎈😶 under 10 처리

diff가 적다면 mean을 활용함이 유리하다고 판단.  
train에서 얻은 bert_date들 중 최대값과 최소값의 평균치를 사용.  

In [18]:
Check_Bert_under10 = Check_Bert_under10.copy()
Check_Bert_under10['max_ata_seconds'] = (Check_Bert_under10['max_ata'] - pd.Timestamp(0)).dt.total_seconds()
Check_Bert_under10['min_ata_seconds'] = (Check_Bert_under10['min_ata'] - pd.Timestamp(0)).dt.total_seconds()
Check_Bert_under10['avg_ata_seconds'] = (Check_Bert_under10['max_ata_seconds'] + Check_Bert_under10['min_ata_seconds']) / 2
Check_Bert_under10['bert_date'] = pd.Timestamp(0) + pd.to_timedelta(np.array(Check_Bert_under10['avg_ata_seconds']), unit='s')

In [19]:
test_weather_under10 = test_weather.merge(Check_Bert_under10[['check_bert','bert_date']], on='check_bert', how='left')

In [20]:
test_weather_under10 = test_weather_under10[test_weather_under10['bert_date'].notnull()]

In [21]:
len(test_weather_under10)

63774

In [22]:
test_weather_under10['ATA'] = pd.to_datetime(test_weather_under10['ATA'])
test_weather_under10['real_ata_date'] = pd.to_datetime(test_weather_under10['ATA']).dt.floor('D')
test_weather_under10['bert_ata_date'] = pd.to_datetime(test_weather_under10['bert_date']).dt.floor('D')
test_weather_under10['real_ata_seconds'] = (test_weather_under10['ATA'] - pd.Timestamp(0)).dt.total_seconds() / 3600
test_weather_under10['bert_ata_seconds'] = (test_weather_under10['bert_date'] - pd.Timestamp(0)).dt.total_seconds() / 3600

예측한 bert_date가 test시점의 ata_date보다 이전이면 후처리 제외.  
train에서 얻은 정보를 기반으로 test에서도 target값이 train의 target.max를 넘지 않을 것이라고 판단하여 제외.  

In [23]:
test_weather_under10 = test_weather_under10[test_weather_under10['real_ata_date'] <= test_weather_under10['bert_ata_date']]

In [24]:
train['CI_HOUR'].max()

2159.130556

In [25]:
test_weather_under10 = test_weather_under10[test_weather_under10['bert_ata_seconds'] <= (test_weather_under10['real_ata_seconds']+2159.130556)]

In [26]:
test_weather_under10['CI_HOUR'] = test_weather_under10['bert_ata_seconds'] - test_weather_under10['real_ata_seconds']

In [27]:
test_weather_under10.loc[test_weather_under10['CI_HOUR'] < 0, 'CI_HOUR'] = 0
test_weather_under10.loc[test_weather_under10['DIST'] == 0, 'CI_HOUR'] = 0

***

# (●'◡'●) 여기에 submission을 넣으세요.

In [28]:
submission = pd.read_csv('../submission/sample_submission.csv')

In [30]:
sub_jun = pd.read_csv('../submission/lgbm_seed_ensem.csv')
sub_woo = pd.read_csv('../submission/final_sub_woo.csv')

두개의 모델을 7대3 으로 산술평균 앙상블 진행

In [31]:
submission['CI_HOUR'] = (0.7 * sub_jun['CI_HOUR']) + (0.3 * sub_woo['CI_HOUR'])

In [32]:
submission.describe()

,CI_HOUR
count,220491.000000
mean,38.638491
std,71.825083
min,0.000000
25%,0.000000
50%,16.419769
75%,52.885908
max,1788.245121


test_weather_under10 조건에 해당하는 row만 submission에서 후처리 진행

In [33]:
submission.loc[submission['SAMPLE_ID'].isin(test_weather_under10['SAMPLE_ID']), 'CI_HOUR'] = test_weather_under10['CI_HOUR'].tolist()

In [34]:
submission.describe()

,CI_HOUR
count,220491.000000
mean,46.442501
std,115.220445
min,0.000000
25%,0.000000
50%,13.301085
75%,53.002307
max,2154.033056


***

# 🎈😶 over 10 처리

해당하는 train의 bert row들을 bert_lst에 저장

diff가 크다면 min 값을 활용함이 유리하다고 판단.  
train에서 얻은 bert_date들 중 최소값의 평균치를 사용.  

In [35]:
Check_Bert_over10['bert_lst'] = None
bert_lst_accumulator = []

for i in tqdm(range(len(Check_Bert_over10))):
    check_b = Check_Bert_over10.iloc[i,0]
    tmp = train_weather[train_weather['check_bert'] == check_b]
    bert_lst_accumulator.append(tmp['bert_date'].tolist())

100%|█████████████████████████████████████████████████████████████████████████████| 1287/1287 [00:11<00:00, 111.74it/s]


In [36]:
Check_Bert_over10['bert_lst'] = bert_lst_accumulator

bert_lst에 위에서 이야기 했던 train의 bert_date들이 들어있음.  
이후 test에 merge

In [37]:
test_weather_over10 = test_weather.merge(Check_Bert_over10[['check_bert','bert_lst']], on='check_bert', how='left')

In [38]:
test_weather_over10 = test_weather_over10[test_weather_over10['bert_lst'].notnull()].copy()

In [39]:
len(test_weather_over10)

4122

***

test 속의 행에 조건을 걸어 test 속에서 열 연산을 통해 effective_bert_lst 생성.  
(조건 : test['ATA'] 가 bert값(여기서의 bert값은 test속에 있는 컬럼)보다 작거나 같고 크기차이가 train['CI_HOUR'].max()보다 크지 않을것)

In [40]:
test_weather_over10['effective_bert_lst'] = None

In [41]:
test_weather_over10['ATA'] = pd.to_datetime(test_weather_over10['ATA'])

In [42]:
effective_bert_lst_accumulator = []

for i in tqdm(range(len(test_weather_over10))):
    real_ata = test_weather_over10.iloc[i, 5]
    bt_lst = test_weather_over10.iloc[i,23]
    
    tmp_lst = []
    
    for j in bt_lst:
        comp_ata = pd.to_datetime(j)
        
        real_ata_date = real_ata.floor('D')
        bert_ata_date = comp_ata.floor('D')
        real_ata_seconds = (real_ata - pd.Timestamp(0)).total_seconds() / 3600
        bert_ata_seconds = (comp_ata - pd.Timestamp(0)).total_seconds() / 3600
        
        if real_ata_date <= bert_ata_date:
            if bert_ata_seconds <= (real_ata_seconds+2159.130556):
                tmp_lst.append(comp_ata)
        
    effective_bert_lst_accumulator.append(tmp_lst)

100%|█████████████████████████████████████████████████████████████████████████████| 4122/4122 [00:06<00:00, 610.28it/s]


In [43]:
test_weather_over10['effective_bert_lst'] = effective_bert_lst_accumulator

***

effective_bert_lst 의 크기가 0이라면 이상한 값들 밖에 없는 것 이기 때문에 후처리에서 제외

In [44]:
test_weather_over10 = test_weather_over10[test_weather_over10['effective_bert_lst'].apply(lambda x: len(x) != 0)].copy()

In [45]:
test_weather_over10['bert_date'] = None

min값만 사용.

In [46]:
for i in tqdm(range(len(test_weather_over10))):
    ata_lst = test_weather_over10.iloc[i, 24]
    avg_sum = 0
#     avg_sum += (max(ata_lst) - Timestamp(0)).total_seconds()
    avg_sum += (min(ata_lst) - Timestamp(0)).total_seconds()
    avg_sum = avg_sum #/2
    average_timestamp = Timestamp(0) + timedelta(seconds=avg_sum)
    test_weather_over10.iloc[i,25] = average_timestamp

100%|███████████████████████████████████████████████████████████████████████████| 3879/3879 [00:00<00:00, 16648.13it/s]


In [47]:
test_weather_over10['bert_date'] = pd.to_datetime(test_weather_over10['bert_date'])

In [48]:
test_weather_over10['real_ata_seconds'] = (test_weather_over10['ATA'] - pd.Timestamp(0)).dt.total_seconds() / 3600
test_weather_over10['bert_ata_seconds'] = (test_weather_over10['bert_date'] - pd.Timestamp(0)).dt.total_seconds() / 3600

In [49]:
test_weather_over10['CI_HOUR'] = test_weather_over10['bert_ata_seconds'] - test_weather_over10['real_ata_seconds']

In [50]:
test_weather_over10.loc[test_weather_over10['CI_HOUR'] < 0, 'CI_HOUR'] = 0
test_weather_over10.loc[test_weather_over10['DIST'] == 0, 'CI_HOUR'] = 0

***

In [51]:
submission.describe()

,CI_HOUR
count,220491.000000
mean,46.442501
std,115.220445
min,0.000000
25%,0.000000
50%,13.301085
75%,53.002307
max,2154.033056


test_weather_over10 조건에 해당하는 row만 submission에서 후처리 진행

In [52]:
submission.loc[submission['SAMPLE_ID'].isin(test_weather_over10['SAMPLE_ID']), 'CI_HOUR'] = test_weather_over10['CI_HOUR'].tolist()

In [53]:
submission.describe()

,CI_HOUR
count,220491.000000
mean,47.242550
std,118.696355
min,0.000000
25%,0.000000
50%,13.172348
75%,53.140983
max,2154.033056


In [54]:
submission.to_csv('../submission/FINAL_7_3_afterprocessing.csv', index = False)

***
***